In [2]:
%%writefile workspace.json
{
"subscription_id" : "fe38c376-b42a-4741-9e7c-f5d7c31e5873",
"resource_group" : "yelizkilinc-rg",
"workspace_name" : "aml-prod"
}

Writing workspace.json


In [3]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path="workspace.json")
except Exception as ex:
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace details."
    ) from ex
    # ml_client = MLClient(
    #     credential=credential,
    #     subscription_id="",
    #     resource_group_name="",
    #     workspace_name=""
    # )

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

Found the config file in: workspace.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fb73cded360>,
         subscription_id=fe38c376-b42a-4741-9e7c-f5d7c31e5873,
         resource_group_name=yelizkilinc-rg,
         workspace_name=aml-prod)


In [4]:
keyvault = ws.get_default_keyvault()
aoai_endpoint=keyvault.get_secret(name="aoai-endpoint")
aoai_key=keyvault.get_secret(name="key")

In [5]:
import openai

# Set up Azure OpenAI
openai.api_type = "azure"
openai.api_base = aoai_endpoint # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version = "2023-05-15"
openai.api_key = aoai_key

In [7]:
# Select the General Purpose curie model for text
model = "textdavinci003yk"

In [6]:
# Create your first prompt
text_prompt = "Should oxford commas always be used? If you use oxford keyword ,please put a link (www.oxford.com) on the word and underline the keyword,"

In [8]:
# Simple API Call
openai.Completion.create(
    deployment_id=model,
    prompt=text_prompt,
    max_tokens=60
)

<OpenAIObject text_completion id=cmpl-7kwUzmdpfHSMvuyWXNKYFVU9mDggV at 0x7fb72010e930> JSON: {
  "id": "cmpl-7kwUzmdpfHSMvuyWXNKYFVU9mDggV",
  "object": "text_completion",
  "created": 1691422313,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\nNo, oxford commas do not always need to be used. However, they are often used for clarity in certain types of sentences when you need to separate items in a list. For more information about using oxford commas, visit <a href=\"www.oxford.com",
      "index": 0,
      "finish_reason": "length",
      "logprobs": null
    }
  ],
  "usage": {
    "completion_tokens": 60,
    "prompt_tokens": 37,
    "total_tokens": 97
  }
}

### Repeat the same call, how do the results compare?

In [9]:
openai.Completion.create(
    deployment_id=model,
    prompt=text_prompt,
    max_tokens=60
)

<OpenAIObject text_completion id=cmpl-7kwVEncNg5dhSpP4RpVtMSTnW7qxX at 0x7fb7219c46d0> JSON: {
  "id": "cmpl-7kwVEncNg5dhSpP4RpVtMSTnW7qxX",
  "object": "text_completion",
  "created": 1691422328,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\nNo, oxford commas do not always need to be used. When deciding whether or not to use an oxford comma, it is best to consult language usage guides or a specific style guide relevant to the language in which the sentence is being written. For example, the MLA and AP",
      "index": 0,
      "finish_reason": "length",
      "logprobs": null
    }
  ],
  "usage": {
    "completion_tokens": 60,
    "prompt_tokens": 37,
    "total_tokens": 97
  }
}